# refining & features engineering

concat train and test

list:
    - date features add cos and sin and holidays
    - lag rolling features 
    - add oil
    - add info store

In [ ]:
## ADD CUTOFF 

In [72]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import datetime
import os

In [2]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales_kaggle/'
df_train = pd.read_csv(path + '/data/trainclean.csv', sep=';')
df_train['date'] = pd.to_datetime(df_train['date'])
df_test = pd.read_csv(path + '/data/testclean.csv', sep=';')
df_test['date'] = pd.to_datetime(df_test['date'])

df = pd.concat([df_train, df_test], axis=0)
df.head()

,ts_id,date,id,bu,famid,sales,onprom
0,1_0,2013-01-02,1782,1,0,2.0,0.0
1,1_0,2013-01-03,3564,1,0,3.0,0.0
2,1_0,2013-01-04,5346,1,0,3.0,0.0
3,1_0,2013-01-05,7128,1,0,5.0,0.0
4,1_0,2013-01-06,8910,1,0,2.0,0.0


### date features

In [4]:
df_f_d = df[['date']].drop_duplicates()
df_f_d['day'] = df_f_d['date'].dt.day
df_f_d['week'] = df_f_d['date'].dt.isocalendar().week
df_f_d['month'] = df_f_d['date'].dt.month
df_f_d['year'] = df_f_d['date'].dt.year
df_f_d['yearmoment'] = (df_f_d['year'] - df_f_d['year'].min())
df_f_d['weekofmonth'] = df_f_d['day'].apply(lambda x: np.ceil(x/7))

df_f_d['dayofweek'] = df_f_d['date'].dt.dayofweek
df_f_d['weekend'] = (df_f_d['dayofweek']>=5)

df_f_d['cosweek'] = np.cos(df_f_d['week'])
df_f_d['sinweek'] = np.sin(df_f_d['week'])

df_f_d.tail()

df_hol_event = pd.read_csv(path + '/data/holidays_events.csv')
hol_ev_keep = ['Holiday', 'Additional', 'Event']
df_hol_event = df_hol_event[df_hol_event['type'].isin(hol_ev_keep)][['date']]
df_hol_event['date'] = pd.to_datetime(df_hol_event['date'])
df_hol_event = df_hol_event.drop_duplicates()

df_hol_event['hol'] = 1

df_f_d_h = df_f_d.merge(df_hol_event, on=['date'], how='left')
df_f_d_h['hol'] = df_f_d_h['hol'].fillna(0).astype(int)
df_f_d_h['hol_before_1'] = df_f_d_h['hol'].shift(1).fillna(0).astype(int)
df_f_d_h['hol_before_2'] = df_f_d_h['hol'].shift(2).fillna(0).astype(int)
df_f_d_h['hol_before_3'] = df_f_d_h['hol'].shift(3).fillna(0).astype(int)
df_f_d_h['hol_after_1'] = df_f_d_h['hol'].shift(-1).fillna(0).astype(int)

df_f_d_h.head()

df_oil = pd.read_csv(path + '/data/oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil.columns = ['date', 'prixoil']
df_f_d_h_o = df_f_d_h.merge(df_oil, on=['date'], how='left')
df_f_d_h_o['prixoil'] = df_f_d_h_o['prixoil'].fillna(method='ffill').fillna(method='bfill')

df_f_d_h_o.to_csv('data/fe/date_holidays_oil.csv', index=False, sep=';')
df_f_d_h_o.head()


df_store = pd.read_csv(path + '/data/stores.csv')

df_store['typeid'] = pd.factorize(df_store['type'])[0]
df_store['cityid'] = pd.factorize(df_store['city'])[0]
df_store['stateid'] = pd.factorize(df_store['state'])[0]

df_store_infos = df_store[['store_nbr', 'city', 'state', 'type', 'typeid', 'cityid', 'stateid', 'cluster']].drop_duplicates()
df_store_infos.to_csv('data/infos/df_store_infos.csv', index=False, sep=';')

df_transaction = pd.read_csv(path + '/data/transactions.csv')
df_transaction.columns = ['date', 'bu', 'transactions']
df_transaction['date'] = pd.to_datetime(df_transaction['date'])

df_store_transactions = df[['date', 'bu']].drop_duplicates() \
    .merge(df_transaction, how='left', on=['date', 'bu'])

df_store_transactions['transactions'] = df_store_transactions['transactions'].fillna(0)

df_store_transactions_hol = df_store_transactions.merge(df_f_d_h[['date', 'hol']], on=['date'], how='left')
df_store_transactions_hol['isclosed'] = 0
df_store_transactions_hol.loc[(df_store_transactions_hol['hol'] == 1) & (df_store_transactions_hol['transactions'] == 0),
                              'isclosed'] = 1

df_store_transactions = df_store_transactions_hol.drop('hol', axis=1)

df_store = df_store[['store_nbr', 'typeid', 'cityid', 'stateid', 'cluster']]
df_store.columns = ['bu', 'typeid', 'cityid', 'stateid', 'cluster']

df_store_transactions = df_store_transactions.merge(df_store, how='left', on=['bu'])
df_store_transactions = df_store_transactions.drop_duplicates()
df_store_transactions.to_csv('data/fe/stores_details_transactions.csv', index=False, sep=';')

df_store_transactions.head()

,date,day,week,month,year,yearmoment,weekofmonth,dayofweek,weekend,cosweek,sinweek
19602,2017-08-27,27,34,8,2017,4,4.0,6,True,-0.84857,0.529083
21384,2017-08-28,28,35,8,2017,4,4.0,0,False,-0.903692,-0.428183
23166,2017-08-29,29,35,8,2017,4,5.0,1,False,-0.903692,-0.428183
24948,2017-08-30,30,35,8,2017,4,5.0,2,False,-0.903692,-0.428183
26730,2017-08-31,31,35,8,2017,4,5.0,3,False,-0.903692,-0.428183


In [69]:
def get_lag_features(sales_df, df_specific_lag, mode_shift=True, shift=16, list_lag_generic=[7]):
    
    start_time = time.time()
    
    lag_df = sales_df.copy()
    
    print('Lag with shift: ', mode_shift)

    if shift:

        print('Specifics lags')

        for ts in tqdm(lag_df['ts_id'].unique()):

            l_lag_spec = df_specific_lag[df_specific_lag['ts_id'] == ts]['lag']
            l_lag_spec = l_lag_spec[l_lag_spec >= shift]
            l_lag_spec = (l_lag_spec - shift)[:2]

            for idx, s_lag in enumerate(l_lag_spec):

                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = np.NaN
                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = \
                    lag_df[lag_df['ts_id'] == ts].groupby(['ts_id'])['sales'].transform(lambda x: x.shift(shift + s_lag))

        print('Generics lags')

        for g_lag in list_lag_generic:
            lag_df.loc[:, 'lag_genreric_' + str(g_lag)] = \
                    lag_df.groupby(['ts_id'])['sales'].transform(lambda x: x.shift(shift + g_lag)) 

    else:

        print('Specifics lags')

        for ts in tqdm(lag_df['ts_id'].unique()):

            l_lag_spec = df_specific_lag[df_specific_lag['ts_id'] == ts]['lag'][:2]

            for idx, s_lag in enumerate(l_lag_spec):

                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = np.NaN
                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = \
                    lag_df[lag_df['ts_id'] == ts].groupby(['ts_id'])['sales'].transform(lambda x: x.shift(s_lag))

        print('Generics lags')

        for g_lag in list_lag_generic:
            lag_df.loc[:, 'lag_genreric_' + str(g_lag)] = \
                    lag_df.groupby(['ts_id'])['sales'].transform(lambda x: x.shift(g_lag))  


    print('%0.2f min: Time for loops' % ((time.time() - start_time) / 60))
    
    return lag_df


def get_rolling_features(sales_df, mode_shift=True, shift=16, list_windows_rolling=[7]):
    
    start_time = time.time()
    
    rol_df = sales_df.copy()
    
    print('Rolling with shift: ', mode_shift)

    if shift:

        for r_win in list_windows_rolling:
            rol_df['rolling_mean_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).mean())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_std_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).std())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_min_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).min())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_max_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).max())

    else:

        for r_win in list_windows_rolling:
            rol_df['rolling_mean_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).mean())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_std_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).std())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_min_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).min())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_max_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).max())


    print('%0.2f min: Time for loops' % ((time.time() - start_time) / 60))
    
    return rol_df

In [9]:
# add cutoff strtegie

In [73]:
list_cutoff = ['2017-07-31', '2017-07-15', '2017-06-29', '2017-06-13']
#list_cutoff = ['2017-08-16']


list_lag_generic = [1, 7, 14, 28, 56, 104, 365]
list_rolling = [7, 14, 30, 60, 180]
horizon = 16

agg_cols =  [
            ['stateid'],
            ['bu'],
            ['cityid'],
            ['famid'],
            ['stateid', 'famid'],
            ['stateid', 'cityid'],
            ['stateid', 'cluster'],
            ['cluster', 'bu'],
            ['cluster']
            ]

In [79]:
for cutoff in tqdm(list_cutoff):
    
    print(cutoff)
    d_cutoff = pd.to_datetime(cutoff)
    day_max = d_cutoff +  datetime.timedelta(days=horizon -1)
    df_cutoff = df[df['date'] <= day_max]
    df_cutoff.loc[df_cutoff['date'] >= d_cutoff, 'sales'] = np.NaN
    print(df_cutoff.date.max())
    
    outdir = path + '/data/fe/cutoff/{}'.format(cutoff)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    df_cutoff.to_csv('data/fe/cutoff/{}/sales_cutoff.csv'.format(cutoff), index=False, sep=';')
    
    
    sales_df = df_cutoff[['ts_id','date', 'sales']]
    df_specific_l = pd.read_csv(path + 'data/specific_lag_ts.csv')
    
    lag_df = get_lag_features(sales_df, df_specific_l, mode_shift=False, shift=0, list_lag_generic=list_lag_generic)
    lag_df.to_csv('data/fe/cutoff/{}/lag_recurisve.csv'.format(cutoff), index=False, sep=';')
    
    rol_df = get_rolling_features(sales_df, mode_shift=False, shift=0, list_windows_rolling=list_rolling)
    rol_df.to_csv('data/fe/cutoff/{}/rol_recurisve.csv'.format(cutoff), index=False, sep=';')
    
    df_cutoff_store = df_cutoff.merge(df_store_transactions, how='left', on=['date', 'bu'])

    for col in agg_cols:
        print('Encoding', col)
        col_name = '_'+'_'.join(col)+'_'
        df_cutoff_store['enc'+col_name+'mean'] = df_cutoff_store.groupby(col)['sales'].transform('mean')
        df_cutoff_store['enc'+col_name+'std'] = df_cutoff_store.groupby(col)['sales'].transform('std')
        df_cutoff_store['enc'+col_name+'min'] = df_cutoff_store.groupby(col)['sales'].transform('min')
        df_cutoff_store['enc'+col_name+'max'] = df_cutoff_store.groupby(col)['sales'].transform('max')
        df_cutoff_store['enc'+col_name+'median'] = df_cutoff_store.groupby(col)['sales'].transform('median')

    keep_cols = [col for col in list(df_cutoff_store) if 'enc_' in col]
    df_encoding_agg = df_cutoff_store[['ts_id', 'date', 'id', 'bu', 'famid'] + keep_cols]

    df_encoding_agg.to_csv('data/fe/cutoff/{}/encoding_agg.csv'.format(cutoff), index=False, sep=';')

  0%|                                                     | 0/4 [00:00<?, ?it/s]

2017-07-31
2017-08-15 00:00:00
Lag with shift:  False
Specifics lags



100%|███████████████████████████████████████| 1729/1729 [12:43<00:00,  2.26it/s]


Generics lags
12.76 min: Time for loops
Rolling with shift:  False
0.14 min: Time for loops
Encoding ['stateid']
Encoding ['bu']
Encoding ['cityid']
Encoding ['famid']
Encoding ['stateid', 'famid']
Encoding ['stateid', 'cityid']
Encoding ['stateid', 'cluster']
Encoding ['cluster', 'bu']
Encoding ['cluster']


 25%|███████████                                 | 1/4 [14:45<44:17, 885.71s/it]

...

 75%|███████████████████████████▊         | 3/4 [15:29:26<6:58:57, 25137.95s/it]

2017-06-13
2017-06-28 00:00:00
Lag with shift:  False
Specifics lags



100%|█████████████████████████████████████| 1728/1728 [9:08:46<00:00, 19.05s/it]


Generics lags
548.81 min: Time for loops
Rolling with shift:  False
0.13 min: Time for loops
Encoding ['stateid']
Encoding ['bu']
Encoding ['cityid']
Encoding ['famid']
Encoding ['stateid', 'famid']
Encoding ['stateid', 'cityid']
Encoding ['stateid', 'cluster']
Encoding ['cluster', 'bu']
Encoding ['cluster']


100%|███████████████████████████████████████| 4/4 [25:00:55<00:00, 22513.90s/it]


In [40]:
horizon = 16
cutoff = '2017-08-16'

In [59]:
pd.to_datetime('2017-06-29') - datetime.timedelta(days=horizon)

Timestamp('2017-06-13 00:00:00')

In [56]:
d_cutoff = pd.to_datetime(cutoff)
day_max = d_cutoff +  datetime.timedelta(days=horizon -1)
df_cutoff = df[df['date'] <= day_max]
df_cutoff.loc[df_cutoff['date'] >= d_cutoff, 'sales'] = np.NaN

In [57]:
df_cutoff.date.max()

Timestamp('2017-07-30 00:00:00')

In [236]:
list_lag_generic = [1, 7, 14, 28, 56, 104, 365]
list_rolling = [7, 14, 30, 60, 180]


sales_df = df_cutoff[['ts_id','date', 'sales']]

df_specific_l = pd.read_csv(path + 'data/specific_lag_ts.csv')
df_specific_l.head()

,ts_id,lag
0,10_0,1
1,10_0,4
2,10_0,7
3,10_0,13
4,10_0,14


In [ ]:
def get_lag_features(sales_df, df_specific_lag, mode_shift=True, shift=16, list_lag_generic=[7]):
    
    start_time = time.time()
    
    lag_df = sales_df.copy()
    
    print('Lag with shift: ', mode_shift)

    if shift:

        print('Specifics lags')

        for ts in tqdm(lag_df['ts_id'].unique()):

            l_lag_spec = df_specific_lag[df_specific_lag['ts_id'] == ts]['lag']
            l_lag_spec = l_lag_spec[l_lag_spec >= shift]
            l_lag_spec = (l_lag_spec - shift)[:2]

            for idx, s_lag in enumerate(l_lag_spec):

                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = np.NaN
                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = \
                    lag_df[lag_df['ts_id'] == ts].groupby(['ts_id'])['sales'].transform(lambda x: x.shift(shift + s_lag))

        print('Generics lags')

        for g_lag in list_lag_generic:
            lag_df.loc[:, 'lag_genreric_' + str(g_lag)] = \
                    lag_df.groupby(['ts_id'])['sales'].transform(lambda x: x.shift(shift + g_lag)) 

    else:

        print('Specifics lags')

        for ts in tqdm(lag_df['ts_id'].unique()):

            l_lag_spec = df_specific_lag[df_specific_lag['ts_id'] == ts]['lag'][:2]

            for idx, s_lag in enumerate(l_lag_spec):

                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = np.NaN
                lag_df.loc[lag_df['ts_id'] == ts, 'lag_specific_' + str(idx)] = \
                    lag_df[lag_df['ts_id'] == ts].groupby(['ts_id'])['sales'].transform(lambda x: x.shift(s_lag))

        print('Generics lags')

        for g_lag in list_lag_generic:
            lag_df.loc[:, 'lag_genreric_' + str(g_lag)] = \
                    lag_df.groupby(['ts_id'])['sales'].transform(lambda x: x.shift(g_lag))  


    print('%0.2f min: Time for loops' % ((time.time() - start_time) / 60))
    
    return lag_df

In [207]:
lag_df = get_lag_features(sales_df, df_specific_l, mode_shift=False, shift=0, list_lag_generic=list_lag_generic)

Lag with shift:  False
Specifics lags


100%|███████████████████████████████████████████████████████████████| 1782/1782 [12:47<00:00,  2.32it/s]


Generics lags
12.82 min: Time for loops


In [216]:
lag_df.columns

Index(['ts_id', 'date', 'sales', 'lag_specific_0', 'lag_specific_1',
       'lag_genreric_1', 'lag_genreric_7', 'lag_genreric_14',
       'lag_genreric_28', 'lag_genreric_56', 'lag_genreric_104',
       'lag_genreric_365'],
      dtype='object')

In [239]:
lag_df.to_csv('data/fe/cutoff/{}/lag_recurisve.csv'.format(cutoff), index=False, sep=';')

In [209]:
def get_rolling_features(sales_df, mode_shift=True, shift=16, list_windows_rolling=[7]):
    
    start_time = time.time()
    
    rol_df = sales_df.copy()
    
    print('Rolling with shift: ', mode_shift)

    if shift:

        for r_win in list_windows_rolling:
            rol_df['rolling_mean_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).mean())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_std_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).std())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_min_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).min())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_max_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(shift).rolling(r_win).max())

    else:

        for r_win in list_windows_rolling:
            rol_df['rolling_mean_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).mean())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_std_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).std())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_min_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).min())
            
        for r_win in list_windows_rolling:
            rol_df['rolling_max_' + str(r_win)] = rol_df.groupby(['ts_id'])['sales'] \
                .transform(lambda x: x.shift(1).rolling(r_win).max())


    print('%0.2f min: Time for loops' % ((time.time() - start_time) / 60))
    
    return rol_df

In [212]:
rol_df = get_rolling_features(sales_df, mode_shift=False, shift=0, list_windows_rolling=list_rolling)

Rolling with shift:  False
0.14 min: Time for loops


In [217]:
rol_df.columns

Index(['ts_id', 'date', 'sales', 'rolling_mean_7', 'rolling_mean_14',
       'rolling_mean_30', 'rolling_mean_60', 'rolling_mean_180',
       'rolling_std_7', 'rolling_std_14', 'rolling_std_30', 'rolling_std_60',
       'rolling_std_180', 'rolling_min_7', 'rolling_min_14', 'rolling_min_30',
       'rolling_min_60', 'rolling_min_180', 'rolling_max_7', 'rolling_max_14',
       'rolling_max_30', 'rolling_max_60', 'rolling_max_180'],
      dtype='object')

In [240]:
rol_df.to_csv('data/fe/cutoff/{}/rol_recurisve.csv'.format(cutoff), index=False, sep=';')

In [20]:
df_cutoff.shape

(2450500, 7)

In [27]:
df_cutoff.groupby(['date', 'ts_id']).size().max()

1

In [24]:
df_store_transactions.columns

Index(['date', 'bu', 'transactions', 'isclosed', 'typeid', 'cityid', 'stateid',
       'cluster'],
      dtype='object')

In [32]:
df_store_transactions.groupby(['date', 'bu']).size().max()

1

In [62]:
df_store_transactions.tail()

,date,bu,transactions,isclosed,typeid,cityid,stateid,cluster
86637,2017-08-31,54,0.0,0,2,21,15,3
86638,2017-08-31,6,0.0,0,0,0,0,13
86639,2017-08-31,7,0.0,0,0,0,0,8
86640,2017-08-31,8,0.0,0,0,0,0,8
86641,2017-08-31,9,0.0,0,1,0,0,6


In [63]:
df_cutoff_store = df_cutoff.merge(df_store_transactions, how='left', on=['date', 'bu'])
df_cutoff_store.tail()

,ts_id,date,id,bu,famid,sales,onprom,transactions,isclosed,typeid,cityid,stateid,cluster
2450495,9_32,2017-08-11,2993759,9,32,NaN,0.0,1848.0,0,1,0,0,6
2450496,9_32,2017-08-12,2995541,9,32,NaN,4.0,1920.0,0,1,0,0,6
2450497,9_32,2017-08-13,2997323,9,32,NaN,0.0,1745.0,0,1,0,0,6
2450498,9_32,2017-08-14,2999105,9,32,NaN,0.0,1971.0,0,1,0,0,6
2450499,9_32,2017-08-15,3000887,9,32,NaN,0.0,2155.0,0,1,0,0,6


In [36]:
df_cutoff_store.groupby(['date', 'ts_id']).size().max()

1

In [37]:
df_cutoff_store.shape

(2450500, 13)

In [58]:
agg_cols =  [
            ['stateid'],
            ['bu'],
            ['cityid'],
            ['famid'],
            ['stateid', 'famid'],
            ['stateid', 'cityid'],
            ['stateid', 'cluster'],
            ['cluster', 'bu'],
            ['cluster']
            ]

for col in agg_cols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    df_cutoff_store['enc'+col_name+'mean'] = df_cutoff_store.groupby(col)['sales'].transform('mean')
    df_cutoff_store['enc'+col_name+'std'] = df_cutoff_store.groupby(col)['sales'].transform('std')
    df_cutoff_store['enc'+col_name+'min'] = df_cutoff_store.groupby(col)['sales'].transform('min')
    df_cutoff_store['enc'+col_name+'max'] = df_cutoff_store.groupby(col)['sales'].transform('max')
    df_cutoff_store['enc'+col_name+'median'] = df_cutoff_store.groupby(col)['sales'].transform('median')
    
keep_cols = [col for col in list(df_cutoff_store) if 'enc_' in col]
df_encoding_agg = df_cutoff_store[['ts_id', 'date', 'id', 'bu', 'famid'] + keep_cols]

Encoding ['stateid']
Encoding ['bu']
Encoding ['cityid']
Encoding ['famid']
Encoding ['stateid', 'famid']
Encoding ['stateid', 'cityid']
Encoding ['stateid', 'cluster']
Encoding ['cluster', 'bu']
Encoding ['cluster']


In [59]:
df_encoding_agg.to_csv('data/fe/cutoff/{}/encoding_agg.csv'.format(cutoff), index=False, sep=';')

In [60]:
df_encoding_agg.isnull().sum()

ts_id                         0
date                          0
id                            0
bu                            0
famid                         0
enc_stateid_mean              0
enc_stateid_std               0
enc_stateid_min               0
enc_stateid_max               0
enc_stateid_median            0
enc_bu_mean                   0
enc_bu_std                    0
enc_bu_min                    0
enc_bu_max                    0
enc_bu_median                 0
enc_cityid_mean               0
enc_cityid_std                0
enc_cityid_min                0
enc_cityid_max                0
enc_cityid_median             0
enc_famid_mean                0
enc_famid_std                 0
enc_famid_min                 0
enc_famid_max                 0
enc_famid_median              0
enc_stateid_famid_mean        0
enc_stateid_famid_std         0
enc_stateid_famid_min         0
enc_stateid_famid_max         0
enc_stateid_famid_median      0
enc_stateid_cityid_mean       0
enc_stat

In [61]:
df_encoding_agg[df_encoding_agg['ts_id'] == '9_28'].tail()

,ts_id,date,id,bu,famid,enc_stateid_mean,enc_stateid_std,enc_stateid_min,enc_stateid_max,enc_stateid_median,...,enc_cluster_bu_mean,enc_cluster_bu_std,enc_cluster_bu_min,enc_cluster_bu_max,enc_cluster_bu_median,enc_cluster_mean,enc_cluster_std,enc_cluster_min,enc_cluster_max,enc_cluster_median
2444477,9_28,2017-08-11,2993755,9,28,638.674692,1650.879758,0.0,124717.0,50.0,...,528.849294,1328.239164,0.0,38422.625,29.93,452.914169,1180.269101,0.0,89576.36,26.0
2444478,9_28,2017-08-12,2995537,9,28,638.674692,1650.879758,0.0,124717.0,50.0,...,528.849294,1328.239164,0.0,38422.625,29.93,452.914169,1180.269101,0.0,89576.36,26.0
2444479,9_28,2017-08-13,2997319,9,28,638.674692,1650.879758,0.0,124717.0,50.0,...,528.849294,1328.239164,0.0,38422.625,29.93,452.914169,1180.269101,0.0,89576.36,26.0
2444480,9_28,2017-08-14,2999101,9,28,638.674692,1650.879758,0.0,124717.0,50.0,...,528.849294,1328.239164,0.0,38422.625,29.93,452.914169,1180.269101,0.0,89576.36,26.0
2444481,9_28,2017-08-15,3000883,9,28,638.674692,1650.879758,0.0,124717.0,50.0,...,528.849294,1328.239164,0.0,38422.625,29.93,452.914169,1180.269101,0.0,89576.36,26.0
